# Eval

In [ ]:
#| default_exp eval

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from stringdale import (
    Define,
    Scope,
    V,
    E,
    Condition,
    draw_nx
)

from frozendict import frozendict
from stringdale.core import  checkLogs
import pytest
import asyncio
from pydantic import BaseModel, ConfigDict

## Design

In [ ]:
"""

Data model

We have a dataset
* containing tests
* each test has the input to the agent
* and the expected output
* test is any object that can be serialized to json
* expected output is a partial trace spec

* partial trace spec is a list of steps
* each step has a name is a dict with accessors and value are how to check them
* names are the node name we expect to see in the trace
* the dict defines what we expect the value to look like


When we run a dataset, we take the input, run the agent, and check the output against the partial trace spec
since the partial trace spec does not 


"""

'\n\nData model\n\nWe have a dataset\n* containing tests\n* each test has the input to the agent\n* and the expected output\n* test is any object that can be serialized to json\n* expected output is a partial trace spec\n\n* partial trace spec is a list of steps\n* each step has a name is a dict with accessors and value are how to check them\n* names are the node name we expect to see in the trace\n* the dict defines what we expect the value to look like\n\n\nWhen we run a dataset, we take the input, run the agent, and check the output against the partial trace spec\nsince the partial trace spec does not \n\n\n'

In [ ]:
class ExpectedTrace:
    pass

class DataPointRun:
    # basically a list of traces, agent input and agent output
    pass



def dynamic_partial_time_warping(trace, expected,comparisons):
    pass
    # this is a parallel dynamic time warping with an abstract distance metric (as opposed to a euclidean embedding)



    # once we have the distance matrix,
    #   we need to somehow get the time breaks (ie clustering the steps into groups)

    # then we need to do alignment to compute the warping path

    # given a set of events we need to figure out how to encode different matches of the set to traces.


    # https://www.geeksforgeeks.org/dynamic-time-warping-dtw-in-time-series/
    

    # TODO idea, instead of sets of parrallel nodes, we could get a partial ordering of the nodes
    # each time we try to match a node, we can only do so on the set of nodes that do not have an unmatched predecessor.



In [ ]:
def collect_traces_from_file(file_path):
    pass

def collect_traces_from_logg_aggregator(logger):
    pass

def run_dataset(agent,dataset,output_dir):
    # for each data point in the dataset
    # run the agent
    # collect the traces into a file
    # return the file path
    pass

def write_comparison_to_file(dataset_run,expected_traces,output_dir):
    # run the comparison and write the results to a file
    pass


In [ ]:
def runs_summary(runs,dir):
    # get the run files and the comparison files
    # get the total metrics per expected node and total
    # make them into a dataframe
    pass

def plot_runs(runs,dir):
    # call runs_summary
    # plot the results
    pass

def check_regressions(runs,dir):
    # get two runs
    # for each input, if the second run is worse than the first, then flag it
    # make a dataframe of the regressions on a whole run basis
    
    # also make a dataframe of the regressions on a per node basis for the runs that regressed.
    pass


In [ ]:
class DataSet:
    pass




## Partial Order Class

In [ ]:
#| export
from typing import TypeVar, Set, List, Optional
from collections import defaultdict

In [ ]:
#| export
T = TypeVar('T')

class PartialOrder:
    def __init__(self):
        # Store the direct less-than relations
        self.direct_less_than: dict[T, set[T]] = defaultdict(set)
        # Store the transitive closure of less-than relations
        self.less_than: dict[T, set[T]] = defaultdict(set)
        
    def add_relation(self, smaller: T, larger: T) -> None:
        """Add a relation where 'smaller' is less than 'larger'"""
        # Add direct relation
        self.direct_less_than[smaller].add(larger)
        
        # Update transitive closure
        # 1. Add direct relation
        self.less_than[smaller].add(larger)
        
        # 2. Add all relations where 'smaller' is less than something that's less than 'larger'
        for x in self.less_than[larger]:
            self.less_than[smaller].add(x)
            
        # 3. Add all relations where something less than 'smaller' is less than 'larger'
        for x in self.less_than:
            if smaller in self.less_than[x]:
                self.less_than[x].add(larger)
                for y in self.less_than[larger]:
                    self.less_than[x].add(y)
    
    def get_predecessors(self, element: T) -> Set[T]:
        """Get all elements that are less than the given element"""
        predecessors = set()
        for potential_pred, successors in self.less_than.items():
            if element in successors:
                predecessors.add(potential_pred)
        return predecessors
    
    def get_successors(self, element: T) -> Set[T]:
        """Get all elements that are greater than the given element"""
        return self.less_than[element]
    
    def get_minimal_elements(self, elements: Set[T]) -> Set[T]:
        """Get all minimal elements from the given set according to the partial order.
        A minimal element has no other elements in the set that are less than it."""
        minimal = set(elements)
        for element in elements:
            # Remove any element that has a predecessor in the set
            predecessors = self.get_predecessors(element)
            if predecessors & elements:  # if there's any overlap with our set
                minimal.remove(element)
        return minimal
    
    def get_maximal_elements(self, elements: Set[T]) -> Set[T]:
        """Get all maximal elements from the given set according to the partial order.
        A maximal element has no other elements in the set that are greater than it."""
        maximal = set(elements)
        for element in elements:
            # Remove any element that has a successor in the set
            successors = self.get_successors(element)
            if successors & elements:  # if there's any overlap with our set
                maximal.remove(element)
        return maximal

In [ ]:
# Example usage
po = PartialOrder()

# Add some relations
po.add_relation(1, 2)
po.add_relation(2, 3)
po.add_relation(4, 5)
po.add_relation(5, "joe")

# Now 1 < 3 should be true due to transitivity
assert po.get_successors(1) == {2, 3}
assert po.get_predecessors(3) == {1, 2}

# Get minimal elements from a set
elements = {1, 2, 3, 4, 5 }
assert po.get_minimal_elements(elements) == {1, 4}

# Get maximal elements from a set
assert po.get_maximal_elements(elements) == {3, 5}


some_new_elements = {1,2,3,4,5,"jack"}
assert po.get_minimal_elements(some_new_elements) == {1,4,"jack"}
assert po.get_maximal_elements(some_new_elements) == {3,5,"jack"}


## Running Example

In [ ]:
example_yaml = """
input:
  content: "hello world"
expected:
  # we give the name of the trace node
  - node_a:
      # we describe what output we expect from the node using accessors as keys
      # the value is what we expect the accessor to return
      b.c: |
        jimmy went
        to the store
      # we can also give a label to the node so we can refer to it later
      # using the $label key
      $label: node_a1

  - node_b:
      # we can give multiple comparisons to the same node, using different accessors
      d.e:
        value: jimmy
        comparison: "regex"
      f.g:
        value: "is a good boy"
        comparison: "chat"
        kwargs:
          case_sensitive: false

  # we can also give a regex to match the node name
  - node_.*:
      .: "store"
      # using the $parallel key we can specify that this node is expected in parallel with the previous node
      # so we do not know which trace will be logged first
      $parallel: true
      $label: node_z

  - node_c:
      b.c: "store"
      # we can specify more complex ordering constraints using before and after using the $label key
      # before and after are either a label or a list of labels
      # in this case we say that node_c should be after node_a1 and before node_z
      $after: node_a1
      $before: node_z
      
"""



In [ ]:
from stringdale.mappings import parse_edge_descriptor

In [ ]:

example_trace = [
    {
        # should be ignored
        "name": "Start",
        "output": "hello world"
    },
    {
        "name": "node_a",
        "output": {'b':{'c':"jimmy went\nto the store\nto buy some milk"}}
    },
    {
        # first option to node c
        "name": "node_c",
        "output": {'b':{'c':"store is good"}}
    },
    {
        # shouldnt match
        "name": "node_a2",
        "output": {'b':{'d':"store"}}
    },
    {
        # first option to node_z
        "name": "node_x",
        "output": "store"
    },
    {
        "name": "node_b",
        "output": {
            'f':{'g':"is a good boy"},
            'd':{'e':"jimmy"}
            }
    },
    {   
        # second option to node c, only relevant if node_* matches to node_y
        "name": "node_c",
        "output": {'b':{'c':"store is good but not good enough"}}
    },
    {
        # second option to node_z
        "name": "node_y",
        "output": "stores"
    },
]

# expected labels: [noda_a1,1 (b) ,node_z,3 (c)] # c needs to be before z
# a:[1], c:[2,6] , b:[5] z:[4,7]

In [ ]:
possible_mappings = {
    frozendict({'node_a1':1,'1':5,'node_z':4,'3':2}), 
    frozendict({'node_a1':1,'1':5,'node_z':7,'3':2}),
    frozendict({'node_a1':1,'1':5,'node_z':7,'3':6}),
}

best_mapping = frozendict({'node_a1':1,'1':5,'node_z':4,'3':2})

## Parsing Expected Traces

In [ ]:
import yaml
from typing import Dict, Any,Optional, Union, List
from pathlib import Path

In [ ]:
class Condition(BaseModel):
    accessor: tuple[str, ...]
    value: Any
    comparison: Optional[str] = None
    kwargs: Dict[str,Any] = {}

class ExpectedTraceStep(BaseModel):
    name: str
    label: Union[str,int]
    conditions: List[Condition]
    before: Optional[List[Union[str,int]]] = None
    after: Optional[List[Union[str,int]]] = None

class ExpectedTrace(BaseModel):
    input: Any
    expected: List[ExpectedTraceStep]

class Trace(BaseModel):
    model_config = ConfigDict(extra='allow')
    name: str
    output: Any

In [ ]:
#| export
def parse_expected_trace_step(yaml_obj: Dict[str,Any],idx:int,labels:List[str]) -> ExpectedTraceStep:
    if len(yaml_obj.keys()) != 1:
        raise SyntaxError(f"Expected a single key in trace step {idx}, got {yaml_obj.keys()}")
    
    name = list(yaml_obj.keys())[0]
    value = yaml_obj[name]
    label = value.pop("$label",None)
    if label is None:
        label = str(idx)

    before = value.pop("$before",list())
    if isinstance(before,str):
        before = [before]
    after = value.pop("$after",list())
    if isinstance(after,str):
        after = [after]
    parallel = value.pop("$parallel",False)

    if parallel and idx == 0:
        raise ValueError(f"Expected trace step {idx} is has $parallel: true, but is the first step")

    if not parallel and len(after) == 0 and idx > 0:
        after.append(labels[-1])
    
    conditions = []
    for accessor,params in value.items():
        if isinstance(params,str):
            params = {"value":params}
        try:
            accessor = parse_edge_descriptor(accessor,start='accessor')
        except Exception as e:
            raise SyntaxError(f"Error parsing accessor {accessor} for step {idx}. Make sure it is formatted correctly") from e
        condition_data ={
            'accessor':accessor,
            **params
        }
        try:
            conditions.append(Condition.model_validate(condition_data))
        except Exception as e:
            raise SyntaxError(f"When parsing condition {value} for step {idx}") from e
    
    return ExpectedTraceStep(name=name,label=label,conditions=conditions,before=before,after=after)
        
    

In [ ]:
yaml_obj = yaml.safe_load(example_yaml)
sub_yaml = yaml_obj['expected'][-1]
sub_yaml

{'node_c': {'b.c': 'store', '$after': 'node_a1', '$before': 'node_z'}}

In [ ]:
parsed =  parse_expected_trace_step(sub_yaml,0,labels=[])
expected = ExpectedTraceStep(name='node_c', label="0", conditions=[Condition(accessor=('b','c'), value='store', comparison=None, kwargs={})], before=['node_z'], after=['node_a1'])
assert parsed == expected, parsed

In [ ]:
parsed_traces = [Trace.model_validate(trace) for trace in example_trace]
parsed_traces

[Trace(name='Start', output='hello world'),
 Trace(name='node_a', output={'b': {'c': 'jimmy went\nto the store\nto buy some milk'}}),
 Trace(name='node_c', output={'b': {'c': 'store is good'}}),
 Trace(name='node_a2', output={'b': {'d': 'store'}}),
 Trace(name='node_x', output='store'),
 Trace(name='node_b', output={'f': {'g': 'is a good boy'}, 'd': {'e': 'jimmy'}}),
 Trace(name='node_c', output={'b': {'c': 'store is good but not good enough'}}),
 Trace(name='node_y', output='stores')]

In [ ]:
#| export
def parse_expected_trace(yaml_str: str) -> ExpectedTrace:
    if isinstance(yaml_str,Path):
        yaml_string = yaml_str.read_text()
    else:
        yaml_string = yaml_str
    
    try:
        yaml_obj = yaml.safe_load(yaml_string)
    except Exception as e:
        raise SyntaxError(f"Error parsing yaml:\n{yaml_string}\n{e}")

    if list(yaml_obj.keys()) != ["input","expected"]:
        raise SyntaxError(f"Expected keys in main scope are 'input' and 'expected', got {yaml_obj.keys()}")

    input = yaml_obj["input"]
    expected = yaml_obj["expected"]

    parsed_steps = []
    labels = []
    for i,expected_step in enumerate(expected):
        try:
            step = parse_expected_trace_step(expected_step,i,labels)
            parsed_steps.append(step)
            labels.append(step.label)
        except Exception as e:
            raise SyntaxError(f"Error parsing expected trace step:\n{expected_step}") from e
    return ExpectedTrace(input=input,expected=parsed_steps)



In [ ]:
from pprint import pprint

In [ ]:
parsed_expected = parse_expected_trace(example_yaml)

expected = parsed_expected.expected
expected

[ExpectedTraceStep(name='node_a', label='node_a1', conditions=[Condition(accessor=('b', 'c'), value='jimmy went\nto the store\n', comparison=None, kwargs={})], before=[], after=[]),
 ExpectedTraceStep(name='node_b', label='1', conditions=[Condition(accessor=('d', 'e'), value='jimmy', comparison='regex', kwargs={}), Condition(accessor=('f', 'g'), value='is a good boy', comparison='chat', kwargs={'case_sensitive': False})], before=[], after=['node_a1']),
 ExpectedTraceStep(name='node_.*', label='node_z', conditions=[Condition(accessor=('.',), value='store', comparison=None, kwargs={})], before=[], after=[]),
 ExpectedTraceStep(name='node_c', label='3', conditions=[Condition(accessor=('b', 'c'), value='store', comparison=None, kwargs={})], before=['node_z'], after=['node_a1'])]

## Mock comparison functions

In [ ]:
def word_overlap(result: str, expected: str,**kwargs) -> float:
    """
    Calculate the distance between result and expected strings based on word overlap.
    Returns a value between 0 and 1, where:
    - 0 means perfect match (all words from result are in expected)
    - 1 means no overlap (no words from result are in expected)
    
    Args:
        result (str): The string to check words from
        expected (str): The string to check words against
        
    Returns:
        float: Distance metric between 0 and 1
    """
    if not isinstance(result,str) or not isinstance(expected,str):
        return np.inf
    # Convert both strings to lowercase and split into words
    result_words = set(result.lower().split())
    expected_words = set(expected.lower().split())
    
    # If result is empty, return 1.0 (maximum distance)
    if not result_words:
        return 1.0
    
    # Calculate overlap
    overlap = len(result_words.intersection(expected_words))
    total = len(result_words)
    
    # Calculate distance (1 - percentage)
    distance = 1.0 - (overlap / total)
    
    return distance

In [ ]:
# Example 1
result = "The quick brown fox"
expected = "The lazy brown dog"
assert word_overlap(result, expected) == 0.5  # Output: 0.5 (2 out of 4 words match)

# Example 2
result = "Hello world"
expected = "Hello there world"
assert word_overlap(result, expected) == 0.0  # Output: 0.0 (all words match)

# Example 3
result = "Python programming"
expected = "Java development"
assert word_overlap(result, expected) == 1.0  # Output: 1.0 (no words match)

In [ ]:
import re

In [ ]:
#| export
def regex(out: str, expected: str,**kwargs) -> float:
    """
    Compare a string against a regex pattern.
    Returns 0 if the regex matches, 1 if it doesn't.
    
    Args:
        out (str): The string to check
        expected (str): The regex pattern to match against
        
    Returns:
        float: 0 if match, 1 if no match
    """
    if not isinstance(out,str) or not isinstance(expected,str):
        return np.inf
    try:
        if re.search(expected, out,flags=re.IGNORECASE) is not None:
            return 0.0
        return 1.0
    except Exception:
        return 1.0


In [ ]:
# Basic matching
assert regex("hello world", "hello") == 0.0  # Simple substring match
assert regex("hello world", "^hello") == 0.0  # Start anchor
assert regex("hello world", "world$") == 0.0  # End anchor
assert regex("hello world", "hello.*world") == 0.0  # Pattern with wildcard

# Non-matching
assert regex("hello world", "goodbye") == 1.0  # No match
assert regex("hello world", "^world") == 1.0  # Wrong position
assert regex("hello world", "hello$") == 1.0  # Wrong position with anchor

# Pattern errors and edge cases
assert regex("hello world", "(unclosed") == 1.0  # Invalid regex pattern
assert regex("hello world", "") == 0.0  # Empty pattern matches anything
assert regex("", ".*") == 0.0  # Empty string matches wildcard
assert regex("", "") == 0.0  # Empty string matches empty pattern

# Case sensitivity
assert regex("Hello World", "hello") == 0.0  # Case-insensitive by default



## Dynamic Partial Time Warping

In [ ]:
import numpy as np
from typing import List, Any, Dict, Callabel
import itertools as it
import re
from stringdale.mappings import access_object

In [ ]:
import logging 
from stringdale.core import checkLogs
logger = logging.getLogger(__name__)

In [ ]:
#| export
def compute_trace_distance(trace,expected,comparisons,default_comparison):

    logger.debug(f"Computing distance for trace {trace} and expected {expected}")
    if not re.search(expected.name, trace.name):
        return np.inf
    
    # check if all accessors are in the trace
    for condition in expected.conditions:
        try: 
            sub_object = access_object(trace.output,condition.accessor)
        except Exception as e:
            return np.inf

    distance = 0
    for condition in expected.conditions:
        condition_func = comparisons.get(condition.comparison, default_comparison)
        output_sub_value = access_object(trace.output,condition.accessor)
        try:
            condition_distance = condition_func(output_sub_value, condition.value, **condition.kwargs)
        except Exception as e:
            raise ValueError(f"Error computing distance for condition {condition} on trace {trace.name}: {e}") from e
        distance += condition_distance
    
    return distance


In [ ]:
comparisons = {
    "regex": regex,
    "word_overlap": word_overlap,
}
default_comparison = word_overlap
# TODO from here, debug the distance function



In [ ]:
import pandas as pd

In [ ]:

#| export
def compute_distance_matrix(
    traces_outputs:List[Any],
    expected_trace:ExpectedTrace,
    comparisons:Dict[str,Callabel],
    default_comparison:Callabel):
    """
    Compute the distance matrix between the traces and the expected traces.

    Args:
        traces_outputs: List[Any], the outputs of the traces
        expected_traces: ExpectedTrace, the expected traces
        comparisons: Dict[str,Callabel], the comparisons to use for the distance matrix
        default_comparison: Callabel, the default comparison to use for the distance matrix
    """
    expected_steps = expected_trace.expected
    distances = np.zeros((len(traces_outputs), len(expected_steps)))
    
    for (i, trace), (j, expected) in it.product(enumerate(traces_outputs), enumerate(expected_steps)):
        distances[i,j] = compute_trace_distance(trace,expected,comparisons,default_comparison)

    row_names = [(i ,trace.name) for i,trace in enumerate(traces_outputs)]
    col_names = [(step.label,step.name) for i,step in enumerate(expected_steps)]

    return pd.DataFrame(distances,index=row_names,columns=col_names)
    

In [ ]:
parsed_traces

[Trace(name='Start', output='hello world'),
 Trace(name='node_a', output={'b': {'c': 'jimmy went\nto the store\nto buy some milk'}}),
 Trace(name='node_c', output={'b': {'c': 'store is good'}}),
 Trace(name='node_a2', output={'b': {'d': 'store'}}),
 Trace(name='node_x', output='store'),
 Trace(name='node_b', output={'f': {'g': 'is a good boy'}, 'd': {'e': 'jimmy'}}),
 Trace(name='node_c', output={'b': {'c': 'store is good but not good enough'}}),
 Trace(name='node_y', output='stores')]

In [ ]:
from numpy import inf

In [ ]:
comparisons = {
    "regex": regex,
    "word_overlap": word_overlap,
}
default_comparison = word_overlap
# with checkLogs():
dist = compute_distance_matrix(parsed_traces,parsed_expected,comparisons,default_comparison)

expected_dist = np.array(   
        [[       inf,        inf,        inf,        inf],
       [0.375     ,        inf,        inf,        inf],
       [       inf,        inf,        inf, 0.66666667],
       [       inf,        inf,        inf,        inf],
       [       inf,        inf, 0.        ,        inf],
       [       inf, 0.        ,        inf,        inf],
       [       inf,        inf,        inf, 0.83333333],
       [       inf,        inf, 1.        ,        inf]]
)

assert np.allclose(dist.values,expected_dist)
dist

,"(node_a1, node_a)","(1, node_b)","(node_z, node_.*)","(3, node_c)"
"(0, Start)",inf,inf,inf,inf
"(1, node_a)",0.375,inf,inf,inf
"(2, node_c)",inf,inf,inf,0.666667
"(3, node_a2)",inf,inf,inf,inf
"(4, node_x)",inf,inf,0.0,inf
"(5, node_b)",inf,0.0,inf,inf
"(6, node_c)",inf,inf,inf,0.833333
"(7, node_y)",inf,inf,1.0,inf


In [ ]:
# TODO from here play with python-constraint2

In [ ]:
parsed_expected.expected

[ExpectedTraceStep(name='node_a', label='node_a1', conditions=[Condition(accessor=('b', 'c'), value='jimmy went\nto the store\n', comparison=None, kwargs={})], before=[], after=[]),
 ExpectedTraceStep(name='node_b', label='1', conditions=[Condition(accessor=('d', 'e'), value='jimmy', comparison='regex', kwargs={}), Condition(accessor=('f', 'g'), value='is a good boy', comparison='chat', kwargs={'case_sensitive': False})], before=[], after=['node_a1']),
 ExpectedTraceStep(name='node_.*', label='node_z', conditions=[Condition(accessor=('.',), value='store', comparison=None, kwargs={})], before=[], after=[]),
 ExpectedTraceStep(name='node_c', label='3', conditions=[Condition(accessor=('b', 'c'), value='store', comparison=None, kwargs={})], before=['node_z'], after=['node_a1'])]

In [ ]:
def int_to_excel_col(n):
    if n < 0:
        raise ValueError("Number must be non-negative")
    
    result = ""
    n += 1  # Adjust because Excel columns start at 1, not 0
    
    while n > 0:
        n -= 1  # Adjust for 0-based indexing
        result = chr(n % 26 + ord('A')).lower() + result
        n //= 26
        
    return result



In [ ]:

assert int_to_excel_col(0) == "a"
assert int_to_excel_col(25) == "z"
assert int_to_excel_col(26) == "aa"
assert int_to_excel_col(27) == "ab"
assert int_to_excel_col(51) == "az"
assert int_to_excel_col(52) == "ba"
assert int_to_excel_col(701) == "zz"
assert int_to_excel_col(702) == "aaa"


In [ ]:
from constraint import Problem,FunctionConstraint
from bidict import bidict

In [ ]:
class LabelToVar():
    def __init__(self):
        self.label_to_var = bidict()
        self.label_to_index = bidict()

    def add_label(self,label:str,idx:int):
        self.label_to_var[label] = int_to_excel_col(idx)
        self.label_to_index[label] = idx

    def get_label(self,col:str) -> str:
        return self.label_to_var.inverse[col]

    def get_index(self,label:str) -> int:
        return self.label_to_index[label]

    def get_col(self,label:str) -> int:
        return self.label_to_var[label]


In [ ]:
label_to_var = LabelToVar()
label_to_var.add_label("x",0)
label_to_var.add_label("y",1)
label_to_var.add_label("z",2)

assert label_to_var.get_col("x") == "a"
assert label_to_var.get_col("y") == "b"
assert label_to_var.get_col("z") == "c"

assert label_to_var.get_label("a") == "x"
assert label_to_var.get_label("b") == "y"
assert label_to_var.get_label("c") == "z"

assert label_to_var.get_index("x") == 0
assert label_to_var.get_index("y") == 1
assert label_to_var.get_index("z") == 2

In [ ]:
dist = dist
expected_traces = parsed_expected.expected
traces = parsed_traces


# get a mapping between expected labels and fresh var names
label_to_var = LabelToVar()
for idx,expected_step in enumerate(parsed_expected.expected):
    label_to_var.add_label(expected_step.label,idx)



In [ ]:
def get_possible_mappings(expected_traces:List[ExpectedTrace],label_to_var:LabelToVar):
    """
    Gets possible mappings between expected traces and actual traces.
    By building a constraint satisfaction problem and solving it.
    """
    p = Problem()
    for col_idx,expected_step in enumerate(parsed_expected.expected):
        viable_trace_row_names = dist.iloc[:,col_idx][dist.iloc[:,col_idx] < np.inf].index.tolist()
        viable_trace_row_nums = [row_name[0] for row_name in viable_trace_row_names]
        var_name = label_to_var.get_col(expected_step.label)
        p.addVariable(var_name,viable_trace_row_nums)
        logger.debug(f"Adding variable {var_name} with domain {viable_trace_row_nums}")

        for before_label in expected_step.before:
            before_var_name = label_to_var.get_col(before_label)
            logger.debug(f"Adding constraint {before_var_name} < {var_name}")
            p.addConstraint(f"{var_name} < {before_var_name}")

        for after_label in expected_step.after:
            after_var_name = label_to_var.get_col(after_label)
            logger.debug(f"Adding constraint {var_name} < {after_var_name}")
            p.addConstraint(f"{after_var_name} < {var_name}")

    # these solutions use colnames    
    solutions = p.getSolutions()
    # invert the colnames back to labels
    labeled_solutions = set(frozendict({label_to_var.get_label(k):v for k,v in sol.items()}) for sol in solutions)
    return labeled_solutions

In [ ]:
from deepdiff import DeepDiff

In [ ]:
with checkLogs():
    labeled_solutions = get_possible_mappings(parsed_expected.expected,label_to_var)

assert DeepDiff(labeled_solutions,possible_mappings) == {}
labeled_solutions

__main__ - DEBUG - Adding variable a with domain [1]
__main__ - DEBUG - Adding variable b with domain [5]
__main__ - DEBUG - Adding constraint b < a
__main__ - DEBUG - Adding variable c with domain [4, 7]
__main__ - DEBUG - Adding variable d with domain [2, 6]
__main__ - DEBUG - Adding constraint c < d
__main__ - DEBUG - Adding constraint d < a


{frozendict.frozendict({'node_a1': 1, '3': 2, '1': 5, 'node_z': 4}),
 frozendict.frozendict({'node_a1': 1, '3': 2, '1': 5, 'node_z': 7}),
 frozendict.frozendict({'node_a1': 1, '3': 6, '1': 5, 'node_z': 7})}

In [ ]:
# TODO from here, make a function that finds best mapping using the dist matrix, possible mappings and dict to var, and then make an e2e function
# that does the matrix construction, label mapping, mapping finding, and then the choosing of the best mapping.

# after that refactor to po time warping notebook and eval notebook

In [ ]:
# TODO from here, make expected of the runnign example
# invert the var names back to labels.

# assert exected

# Factor into po time warping notebook
# and have the higher order code in the eval notebook

In [ ]:
def find_warping_path(distances:np.ndarray,po:PartialOrder):
    """
    Find the warping path between the traces and the expected traces.

    Args:
        distances: np.ndarray, the distance matrix between the traces and the expected traces
        po: PartialOrder, the partial order of the expected traces, assumed po to be over excepted traces indices

    """
    # find the minimal elements of the expected traces
    minimal_elements = po.get_minimal_elements(set(range(len(expected_traces))))

    # build the matching with poset constraints as a CSP problem using python-constraint2
    # add make sure we give each expected trace the domain over elements that is can actually match to
    # so no inf distances

    # iterate over all solution and use the distance matrix to find the best match
    pass
    

In [ ]:
print(example_yaml)


input:
  content: "hello world"
expected:
  - node_a:
      b.c: |
        jimmy went
        to the store
      $label: node_a1

  - node_b:
      d.e:
        value: jimmy
        comparison: "regex"
      f.g:
        value: "is a good boy"
        comparison: "chat"
        kwargs:
          case_sensitive: false

  - node_.*:
      .: "store"
      $parallel: true

  - node_c:
      b.c: "store"
      $before: node_a1



In [ ]:
# TODO 

D(i,j,hist) = 

In [ ]:
# TODO from here, make an example of trace and expected trace with length, equality and regex matching

In [ ]:
# TODO, get a list of traces, a dict of the exected traces (maybe keys by label if exists or by index), and a partial ordering of the expected traces

# compute the distance matrix between the traces and the expected traces

# make the warping path, but rather than a numberic matrix, we need a mapping of who was mapped to who.

In [ ]:
# plot dynamic warping
# we make a plot like https://pypi.org/project/dtaidistance/ maybe

In [ ]:
yaml.safe_load(example_yaml)


[{'input': {'content': 'hello world'},
  'expected': [{'node_a': {'b.c': 'jimmy went\nto the store\n'}},
   {'node_b': {'d.e': {'value': 'jimmy', 'comparison': 'regex'},
     'f.g': {'value': 'is a good boy',
      'comparison': 'chat',
      'kwargs': {'case_sensitive': False}}}},
   {'node_.*': {'.': 'store', '$parallel': True}}]},
 {'input': {'content': 'Go to the store and buy some milk'},
  'expected': [{'node_a': {'b.c': 'store'}}]}]

## Experiment runs

In [ ]:
# TODO start with directories of files with traces.
# here we just run the agent on the input and collect the traces to files
# Later, add a way to customize the runs from a logger or something 
# I think the best way would be to be able to turn the logs into a dataset file and work on it locally.

## Experiment scoring

In [ ]:
# here we use the DPTW to match each trace to an expected trace
# than we have multiple scores
    # total distance, 
    # total distance per expected trace, 
    # coverage (percent of nodes expected), 
    # time coverage (percent of time of nodes expected), used to ignore nodes with no logic


# this experiment object can be dumped into a directory


## Regression detection

In [ ]:

## Regression detection
# here we just compare the runs to each other


## export

In [ ]:
# |hide
import nbdev; nbdev.nbdev_export()